In [54]:
# 관련 영상 정보들을 수집하여 저장하는 파일 

from config import Config
from mysql import connector
import requests
import time
from datetime import datetime
import re
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

def run_query(query):
    conn = connector.connect(**Config) 
    with conn.cursor() as cur:
        cur.execute(query)
        rows= cur.fetchall()
    conn.commit()
    conn.close()

    return rows


music = []
title = []
uploader = []
thumbnail = []
href = []
upload_date = []


options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

driver = webdriver.Chrome(options=options)
url = 'https://www.youtube.com/playlist?list=PL_Cqw69_m_ywY88NJ5SlpuaO_rmpKXO0D'
driver.get(url=url)

for i in range(10) :
    driver.find_element(By.CSS_SELECTOR, 'body').send_keys(Keys.PAGE_DOWN)

html = driver.page_source
soup = bs(html, 'html.parser')

# 썸네일 이미지 리스트

thumbnail_list = soup.find_all('img','yt-core-image--fill-parent-height yt-core-image--fill-parent-width yt-core-image yt-core-image--content-mode-scale-aspect-fill yt-core-image--loaded')
title_wrapper = soup.find_all('a',{'id' : 'video-title'})
for wrap,thumb in zip(title_wrapper, thumbnail_list) :
    if 'Official' in wrap.get_text() : ## Official 영상인 경우에만
        _thumbnail = thumb['src']
        _title = wrap.get_text().replace('\n', '').strip()
        _music = ','.join(re.findall(r"'(.*?)'", _title))
        _href = 'https://youtu.be/' + re.search(r'watch\?v=(.*?)&', wrap['href']).group(1)

        # 얻은 링크에서 업로드일 추출
        sub_driver = webdriver.Chrome(options=options)
        sub_driver.get(url=_href)
        time.sleep(1)
        sub_driver.find_element(By.XPATH, '//*[@id="expand"]').click()
        sub_html = sub_driver.page_source
        sub_soup = bs(sub_html, 'html.parser')
        _date = sub_soup.find_all('span', 'style-scope yt-formatted-string bold')[2].get_text()
        _date = _date.replace("최초 공개: ","")
        _date = _date.replace(" ","-")
        _date = _date.replace(".","")
        sub_driver.quit()

        title.append(_title)  # 영상제목저장
        music.append(_music)  # 음악이름저장
        href.append(_href)    # 영상링크저장
        upload_date.append(_date)   #업로드일저장
        thumbnail.append(_thumbnail)  # 썸네일저장

    print(f"...{_music} 실행 완료...")

query = f'INSERT INTO contents_table VALUES '
for i in range(len(title)) :
    query += f'(NULL, "{music[i]}", "{title[i]}", "HYBE LABELS", "{href[i]}", "{upload_date[i]}", "{thumbnail[i]}", "뮤직비디오"), \n'
query = query[:-3]
run_query(query)


...ASAP 실행 완료...
...ETA 실행 완료...
...ETA 실행 완료...
...Cool With You 실행 완료...
...Cool With You,Get Up 실행 완료...
...Cool With You 실행 완료...
...Super Shy 실행 완료...
...New Jeans 실행 완료...
...New Jeans 실행 완료...
...New Jeans 실행 완료...
...OMG 실행 완료...
...OMG 실행 완료...
...OMG 실행 완료...
...OMG 실행 완료...
...Ditto 실행 완료...
...Ditto 실행 완료...
...Ditto 실행 완료...
...Hype Boy 실행 완료...
...Hype Boy 실행 완료...
...Cookie 실행 완료...
...Cookie 실행 완료...
...Hurt 실행 완료...
...Hype Boy 실행 완료...
...Hype Boy 실행 완료...
...Hype Boy 실행 완료...
...Hype Boy 실행 완료...
...Hype Boy 실행 완료...
...Attention 실행 완료...
...Attention 실행 완료...


[]